# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Measure-Dynamic-Functional-Connectivity" data-toc-modified-id="Measure-Dynamic-Functional-Connectivity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Measure Dynamic Functional Connectivity</a></div><div class="lev2 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Load-CoreData" data-toc-modified-id="Load-CoreData-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load CoreData</a></div><div class="lev2 toc-item"><a href="#Compute-Functional-Connectivity" data-toc-modified-id="Compute-Functional-Connectivity-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Compute Functional Connectivity</a></div><div class="lev3 toc-item"><a href="#Functional-Connectivity-FuncDef" data-toc-modified-id="Functional-Connectivity-FuncDef-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Functional Connectivity FuncDef</a></div><div class="lev3 toc-item"><a href="#Process-Navon" data-toc-modified-id="Process-Navon-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Process Navon</a></div><div class="lev3 toc-item"><a href="#Process-Stroop" data-toc-modified-id="Process-Stroop-133"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Process Stroop</a></div><div class="lev2 toc-item"><a href="#Checking-Correlation-Biases" data-toc-modified-id="Checking-Correlation-Biases-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Checking Correlation Biases</a></div><div class="lev3 toc-item"><a href="#Across-Subjects" data-toc-modified-id="Across-Subjects-141"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Across Subjects</a></div><div class="lev3 toc-item"><a href="#Positive-vs-Negative" data-toc-modified-id="Positive-vs-Negative-142"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Positive vs Negative</a></div><div class="lev3 toc-item"><a href="#Fixation-vs-Task" data-toc-modified-id="Fixation-vs-Task-143"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Fixation vs Task</a></div><div class="lev3 toc-item"><a href="#Within-Task-(Hi-vs-Lo)" data-toc-modified-id="Within-Task-(Hi-vs-Lo)-144"><span class="toc-item-num">1.4.4&nbsp;&nbsp;</span>Within Task (Hi vs Lo)</a></div><div class="lev3 toc-item"><a href="#Between-Task-(Stroop-vs-Navon)" data-toc-modified-id="Between-Task-(Stroop-vs-Navon)-145"><span class="toc-item-num">1.4.5&nbsp;&nbsp;</span>Between Task (Stroop vs Navon)</a></div><div class="lev1 toc-item"><a href="#System-Level-Connectivity" data-toc-modified-id="System-Level-Connectivity-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>System-Level Connectivity</a></div><div class="lev2 toc-item"><a href="#Assign-Lausanne-to-Yeo-Systems" data-toc-modified-id="Assign-Lausanne-to-Yeo-Systems-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Assign Lausanne to Yeo Systems</a></div><div class="lev2 toc-item"><a href="#System-Level-Adjacency-Matrices" data-toc-modified-id="System-Level-Adjacency-Matrices-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>System-Level Adjacency Matrices</a></div><div class="lev3 toc-item"><a href="#Plot-Population-Average-Adjacency-Matrices-(Expr-+-Pos/Neg)" data-toc-modified-id="Plot-Population-Average-Adjacency-Matrices-(Expr-+-Pos/Neg)-221"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Plot Population Average Adjacency Matrices (Expr + Pos/Neg)</a></div><div class="lev3 toc-item"><a href="#Construct-System-Adjacency-Matrices" data-toc-modified-id="Construct-System-Adjacency-Matrices-222"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Construct System Adjacency Matrices</a></div><div class="lev2 toc-item"><a href="#Within/Between-Task-Contrasts" data-toc-modified-id="Within/Between-Task-Contrasts-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Within/Between Task Contrasts</a></div><div class="lev2 toc-item"><a href="#Task-Performance" data-toc-modified-id="Task-Performance-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Task Performance</a></div>

# Measure Dynamic Functional Connectivity

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import scipy.io as io
import h5py
import matplotlib
import matplotlib.pyplot as plt

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(matplotlib.rcParams)

path_Remotes = '/Users/akhambhati/Remotes'
path_CoreData = path_Remotes + '/CORE.fMRI_cogcontrol.medaglia'
path_PeriphData = path_Remotes + '/RSRCH.NMF_CogControl'
path_ExpData = path_PeriphData + '/e01-FuncNetw'
path_AtlasData = path_Remotes + '/CORE.MRI_Atlases'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

## Load CoreData

In [ ]:
# Load BOLD
df_navon = io.loadmat('{}/NavonBlockedSeriesScale125.mat'.format(path_CoreData), struct_as_record=False)
df_stroop = io.loadmat('{}/StroopBlockedSeriesScale125.mat'.format(path_CoreData), struct_as_record=False)

n_subj = 28
n_fix_block = 12 # Disregard the final fixation block
n_tsk_block = 6
n_roi = 262
bad_roi = [242]
n_good_roi = n_roi-len(bad_roi)

# Load Motion Data
df_motion = {'Stroop': io.loadmat('{}/StroopMove.mat'.format(path_CoreData))['move'][:, 0],
             'Navon': io.loadmat('{}/NavonMove.mat'.format(path_CoreData))['move'][:, 0]}

# Load Behavioral Data
df_blk = io.loadmat('{}/BlockwiseDataCorrectTrialsOnly.mat'.format(path_CoreData))
bad_subj_ix = [1, 6]
good_subj_ix = np.setdiff1d(np.arange(n_subj+2), bad_subj_ix)
df_perf = {'Stroop': {'lo': {'accuracy': df_blk['StroopData'][good_subj_ix, 1, :],
                             'meanRT': df_blk['StroopData'][good_subj_ix, 4, :],
                             'medianRT': df_blk['StroopData'][good_subj_ix, 5, :]},
                      'hi': {'accuracy': df_blk['StroopData'][good_subj_ix, 0, :],
                             'meanRT': df_blk['StroopData'][good_subj_ix, 2, :],
                             'medianRT': df_blk['StroopData'][good_subj_ix, 3, :]}
                     },
           'Navon' : {'lo': {'accuracy': df_blk['NavonData'][good_subj_ix, 1, :],
                             'meanRT': df_blk['NavonData'][good_subj_ix, 4, :],
                             'medianRT': df_blk['NavonData'][good_subj_ix, 5, :]},
                      'hi': {'accuracy': df_blk['NavonData'][good_subj_ix, 0, :],
                             'meanRT': df_blk['NavonData'][good_subj_ix, 2, :],
                             'medianRT': df_blk['NavonData'][good_subj_ix, 3, :]}
                     }
          }

## Compute Functional Connectivity

### Functional Connectivity FuncDef

In [ ]:
def comp_fconn(bold, alpha=0.05, dependent=False):
    n_roi, n_tr = bold.shape
    
    adj = np.arctanh(np.corrcoef(bold))
    cfg_vec = convert_adj_matr_to_cfg_matr(adj.reshape(-1, n_roi, n_roi))[0, :]
    
    # Separate edges based on sign
    cfg_vec_pos = cfg_vec.copy()
    cfg_vec_pos[cfg_vec_pos < 0] = 0
    
    cfg_vec_neg = -1*cfg_vec.copy()
    cfg_vec_neg[cfg_vec_neg < 0] = 0
    
    # Normalize the energy of the edges
    energy = np.sqrt(np.sum(cfg_vec_pos**2))
    if np.max(cfg_vec_pos) > 0:
        cfg_vec_pos /= energy

    energy = np.sqrt(np.sum(cfg_vec_neg**2))    
    if np.max(cfg_vec_neg) > 0:
        cfg_vec_neg /= energy

    adj_pos = convert_conn_vec_to_adj_matr(cfg_vec_pos)
    adj_neg = convert_conn_vec_to_adj_matr(cfg_vec_neg)
    
    return adj_pos, adj_neg

### Process Navon

In [ ]:
for subj_id in xrange(n_subj):
    proc_item = '{}/Subject_{}.Navon'.format(path_ExpData, subj_id)
    print(proc_item)
    
    adj_dict = {'lo': {'fix': {'pos': np.zeros((n_tsk_block, n_good_roi, n_good_roi)),
                               'neg': np.zeros((n_tsk_block, n_good_roi, n_good_roi))},
                       'task': {'pos': np.zeros((n_tsk_block, n_good_roi, n_good_roi)),
                                'neg': np.zeros((n_tsk_block, n_good_roi, n_good_roi))}
                      },
                'hi': {'fix': {'pos': np.zeros((n_tsk_block, n_good_roi, n_good_roi)),
                               'neg': np.zeros((n_tsk_block, n_good_roi, n_good_roi))},
                       'task': {'pos': np.zeros((n_tsk_block, n_good_roi, n_good_roi)),
                                'neg': np.zeros((n_tsk_block, n_good_roi, n_good_roi))},
                      }}
    
    # Process Fixation Blocks
    cnt = 0
    for fix_block in xrange(n_fix_block):
        data = np.array(df_navon['data'][subj_id][fix_block].NFix, dtype='f').T
        data = data[np.setdiff1d(np.arange(n_roi), bad_roi), :]
        
        if (fix_block % 2) == 0:
            adj_dict['lo']['fix']['pos'][cnt, :, :], adj_dict['lo']['fix']['neg'][cnt, :, :] = comp_fconn(data)
            
        if (fix_block % 2) == 1:
            adj_dict['hi']['fix']['pos'][cnt, :, :], adj_dict['hi']['fix']['neg'][cnt, :, :] = comp_fconn(data)
            cnt += 1
            
    # Process Task Blocks
    cnt = 0
    for tsk_block in xrange(n_tsk_block):
        
        # Low demand
        data = np.array(df_navon['data'][subj_id][tsk_block].NS, dtype='f').T
        data = data[np.setdiff1d(np.arange(n_roi), bad_roi), :]
        adj_dict['lo']['task']['pos'][cnt, :, :], adj_dict['lo']['task']['neg'][cnt, :, :] = comp_fconn(data)

        # High demand        
        data = np.array(df_navon['data'][subj_id][tsk_block].S, dtype='f').T
        data = data[np.setdiff1d(np.arange(n_roi), bad_roi), :]
        adj_dict['hi']['task']['pos'][cnt, :, :], adj_dict['hi']['task']['neg'][cnt, :, :] = comp_fconn(data)
        
        cnt += 1

        
    np.savez(proc_item, adj_dict=adj_dict)

### Process Stroop

In [ ]:
for subj_id in xrange(n_subj):
    proc_item = '{}/Subject_{}.Stroop'.format(path_ExpData, subj_id)
    print(proc_item)
    
    adj_dict = {'lo': {'fix': {'pos': np.zeros((n_tsk_block, n_good_roi, n_good_roi)),
                               'neg': np.zeros((n_tsk_block, n_good_roi, n_good_roi))},
                       'task': {'pos': np.zeros((n_tsk_block, n_good_roi, n_good_roi)),
                                'neg': np.zeros((n_tsk_block, n_good_roi, n_good_roi))}
                      },
                'hi': {'fix': {'pos': np.zeros((n_tsk_block, n_good_roi, n_good_roi)),
                               'neg': np.zeros((n_tsk_block, n_good_roi, n_good_roi))},
                       'task': {'pos': np.zeros((n_tsk_block, n_good_roi, n_good_roi)),
                                'neg': np.zeros((n_tsk_block, n_good_roi, n_good_roi))},
                      }}
    
    # Process Fixation Blocks
    cnt = 0
    for fix_block in xrange(n_fix_block):
        data = np.array(df_stroop['data'][subj_id][fix_block].SFix, dtype='f').T
        data = data[np.setdiff1d(np.arange(n_roi), bad_roi), :]
        
        if (fix_block % 2) == 0:
            adj_dict['lo']['fix']['pos'][cnt, :, :], adj_dict['lo']['fix']['neg'][cnt, :, :] = comp_fconn(data)
            
        if (fix_block % 2) == 1:
            adj_dict['hi']['fix']['pos'][cnt, :, :], adj_dict['hi']['fix']['neg'][cnt, :, :] = comp_fconn(data)
            cnt += 1
            
    # Process Task Blocks
    cnt = 0
    for tsk_block in xrange(n_tsk_block):
        
        # Low demand
        data = np.array(df_stroop['data'][subj_id][tsk_block].IE, dtype='f').T
        data = data[np.setdiff1d(np.arange(n_roi), bad_roi), :]
        adj_dict['lo']['task']['pos'][cnt, :, :], adj_dict['lo']['task']['neg'][cnt, :, :] = comp_fconn(data)

        # High demand        
        data = np.array(df_stroop['data'][subj_id][tsk_block].E, dtype='f').T
        data = data[np.setdiff1d(np.arange(n_roi), bad_roi), :]
        adj_dict['hi']['task']['pos'][cnt, :, :], adj_dict['hi']['task']['neg'][cnt, :, :] = comp_fconn(data)
        
        cnt += 1
        
        
    np.savez(proc_item, adj_dict=adj_dict)

## Checking Correlation Biases

### Across Subjects

In [ ]:
for cor_id in ['pos', 'neg']:
    mcs = []
    motion = []
    perf = []
    df_stroop_list = glob.glob('{}/*.Stroop.npz'.format(path_ExpData))
    df_navon_list = glob.glob('{}/*.Navon.npz'.format(path_ExpData))
    for df_stroop_subj, df_navon_subj in zip(df_stroop_list, df_navon_list):
        subj_id = int(df_stroop_subj.split('/')[-1].split('.')[0].split('_')[1])
        
        motion.append(df_motion['Stroop'][subj_id] + df_motion['Navon'][subj_id])
        perf.append(df_perf['Stroop']['lo']['medianRT'][subj_id, :].mean() +
                    df_perf['Stroop']['hi']['medianRT'][subj_id, :].mean() +
                    df_perf['Navon']['lo']['medianRT'][subj_id, :].mean() +
                    df_perf['Navon']['hi']['medianRT'][subj_id, :].mean())
        
        mcs_subj = []
        for df_expr_subj in [df_stroop_subj, df_navon_subj]:

            for cnd_id in ['lo', 'hi']:
                for tsk_id in ['fix', 'task']:
                    
                    adj_subj = np.load(df_expr_subj)['adj_dict'][()]
                    adj_subj_mean = adj_subj[cnd_id][tsk_id][cor_id].mean(axis=0)
                    n_roi = adj_subj_mean.shape[0]               
                    cfg_subj_mean = convert_adj_matr_to_cfg_matr(adj_subj_mean.reshape(-1, n_roi, n_roi))[0, :]

                    mcs_subj.append(np.mean(cfg_subj_mean))
        mcs.append(mcs_subj)

        
    print(stats.f_oneway(*mcs))
    
    mcs = np.array(mcs)
    print('Motion Correlation: ', stats.pearsonr(mcs.mean(axis=1), motion))
    print('Performance Correlation: ', stats.pearsonr(mcs.mean(axis=1), perf))

    ### Plot
    plt.figure(figsize=(2,2))
    ax = plt.subplot(111)
    
    mcs_reord = mcs[np.argsort(mcs.mean(axis=1))[::-1], :]
    bp = ax.boxplot(mcs_reord.T, patch_artist=True)
    Echobase.Plotting.fig_format.set_box_color(bp, [0.0, 0.0, 0.0], [[0.2, 0.2, 0.2] for iii in xrange(mcs_reord.shape[0])])

    ax.set_ylim(ymin=0)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    ax.set_xticklabels([])
    ax.set_xlabel('Subjects')
    ax.set_ylabel('Mean Edge Strength')
    ax.set_title(cor_id)
    plt.show()

### Positive vs Negative

In [ ]:
mcs = [[], []]
df_stroop_list = glob.glob('{}/*.Stroop.npz'.format(path_ExpData))
df_navon_list = glob.glob('{}/*.Navon.npz'.format(path_ExpData))
for df_stroop_subj, df_navon_subj in zip(df_stroop_list, df_navon_list):
    for cor_ii, cor_id in enumerate(['pos', 'neg']):
        
        mcs_tmp = []
        for df_expr_subj in [df_stroop_subj, df_navon_subj]:
            for cnd_id in ['lo', 'hi']:
                for tsk_id in ['fix', 'task']:
                    
                    adj_subj = np.load(df_expr_subj)['adj_dict'][()]
                    adj_subj_mean = adj_subj[cnd_id][tsk_id][cor_id].mean(axis=0)
                    n_roi = adj_subj_mean.shape[0]               
                    cfg_subj_mean = convert_adj_matr_to_cfg_matr(adj_subj_mean.reshape(-1, n_roi, n_roi))[0, :]

                    mcs_tmp.append(np.mean(cfg_subj_mean))
        mcs[cor_ii].append(np.mean(mcs_tmp))

        
print(stats.ttest_rel(mcs[0], mcs[1]))        


### Plot
plt.figure(figsize=(2,2))
ax = plt.subplot(111)
bp = ax.boxplot(mcs, patch_artist=True)
Echobase.Plotting.fig_format.set_box_color(bp, [0.0, 0.0, 0.0], [[0.2, 0.2, 0.2] for iii in xrange(2)])

ax.set_ylim(ymin=0)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xticklabels(['Positive', 'Negative'], fontsize=5.0) 

ax.set_ylabel('Mean Edge Strength')
plt.show()

### Fixation vs Task

In [ ]:
for cor_id in ['pos', 'neg']:
    
    mcs = [[], []]
    df_stroop_list = glob.glob('{}/*.Stroop.npz'.format(path_ExpData))
    df_navon_list = glob.glob('{}/*.Navon.npz'.format(path_ExpData))
    for df_stroop_subj, df_navon_subj in zip(df_stroop_list, df_navon_list):
        for tsk_ii, tsk_id in enumerate(['fix', 'task']):

            mcs_tmp = []
            for df_expr_subj in [df_stroop_subj, df_navon_subj]:
                for cnd_id in ['lo', 'hi']:

                    adj_subj = np.load(df_expr_subj)['adj_dict'][()]
                    adj_subj_mean = adj_subj[cnd_id][tsk_id][cor_id].mean(axis=0)
                    n_roi = adj_subj_mean.shape[0]               
                    cfg_subj_mean = convert_adj_matr_to_cfg_matr(adj_subj_mean.reshape(-1, n_roi, n_roi))[0, :]

                    mcs_tmp.append(np.mean(cfg_subj_mean))
            mcs[tsk_ii].append(np.mean(mcs_tmp))


    print(stats.ttest_rel(mcs[0], mcs[1]))        


    ### Plot
    plt.figure(figsize=(2,2))
    ax = plt.subplot(111)
    bp = ax.boxplot(mcs, patch_artist=True)
    Echobase.Plotting.fig_format.set_box_color(bp, [0.0, 0.0, 0.0], [[0.2, 0.2, 0.2] for iii in xrange(2)])

    ax.set_ylim(ymin=0)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticklabels(['Fixation', 'Task'], fontsize=5.0)        

    ax.set_ylabel('Mean Edge Strength')
    ax.set_title(cor_id)
    plt.show()

### Within Task (Hi vs Lo)

In [ ]:
for expr_id in ['Stroop', 'Navon']:
    df_list = glob.glob('{}/*.{}.npz'.format(path_ExpData, expr_id))

    for cor_id in ['pos', 'neg']:
        
        mcs = [[], []]    
        for df_subj in df_list:
            for cnd_ii, cnd_id in enumerate(['lo', 'hi']):

                adj_subj = np.load(df_subj)['adj_dict'][()]
                adj_subj_mean = adj_subj[cnd_id]['task'][cor_id].mean(axis=0)
                n_roi = adj_subj_mean.shape[0]               
                cfg_subj_mean = convert_adj_matr_to_cfg_matr(adj_subj_mean.reshape(-1, n_roi, n_roi))[0, :]

                mcs[cnd_ii].append(np.mean(cfg_subj_mean))
            
        print(expr_id, stats.ttest_rel(mcs[0], mcs[1]))  
    

        ### Plot
        plt.figure(figsize=(2,2))
        ax = plt.subplot(111)
        bp = ax.boxplot(mcs, patch_artist=True)
        Echobase.Plotting.fig_format.set_box_color(bp, [0.0, 0.0, 0.0], [[0.2, 0.2, 0.2] for iii in xrange(2)])

        ax.set_ylim(ymin=0)    
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        ax.set_xticklabels(['Lo Demand', 'Hi Demand'], fontsize=5.0)        

        ax.set_ylabel('Mean Edge Strength')
        ax.set_title('{}-{}'.format(expr_id, cor_id))
        plt.show()    

### Between Task (Stroop vs Navon)

In [ ]:
mcs = [[], []]
df_stroop_list = glob.glob('{}/*.Stroop.npz'.format(path_ExpData))
df_navon_list = glob.glob('{}/*.Navon.npz'.format(path_ExpData))
for df_stroop_subj, df_navon_subj in zip(df_stroop_list, df_navon_list):
    for expr_ii, df_expr_subj in enumerate([df_stroop_subj, df_navon_subj]):    
        
        mcs_tmp = []
        for cnd_id in ['lo', 'hi']:
            for cor_id in ['pos', 'neg']:

                adj_subj = np.load(df_expr_subj)['adj_dict'][()]
                adj_subj_mean = adj_subj[cnd_id]['task'][cor_id].mean(axis=0)
                n_roi = adj_subj_mean.shape[0]               
                cfg_subj_mean = convert_adj_matr_to_cfg_matr(adj_subj_mean.reshape(-1, n_roi, n_roi))[0, :]

                mcs_tmp.append(np.mean(cfg_subj_mean))
        mcs[expr_ii].append(np.mean(mcs_tmp))

        
print(stats.ttest_rel(mcs[0], mcs[1]))        


### Plot
plt.figure(figsize=(2,2))
ax = plt.subplot(111)
bp = ax.boxplot(mcs, patch_artist=True)
Echobase.Plotting.fig_format.set_box_color(bp, [0.0, 0.0, 0.0], [[0.2, 0.2, 0.2] for iii in xrange(2)])

ax.set_ylim(ymin=0)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xticklabels(['Stroop', 'Navon'], fontsize=5.0)        

ax.set_ylabel('Mean Edge Strength')
plt.show()

# System-Level Connectivity

## Assign Lausanne to Yeo Systems

In [ ]:
import nibabel as nib

df_yeo_atlas = nib.load('{}/Yeo_JNeurophysiol11_MNI152/Yeo2011_7Networks_MNI152_FreeSurferConformed1mm_LiberalMask.nii.gz'.format(path_AtlasData))
yeo_matr = df_yeo_atlas.get_data()[..., 0]
yeo_roi = np.unique(yeo_matr)[1:]
yeo_names = ['VIS', 'SMN', 'DAN', 'VAN', 'LIM', 'FPN', 'DMN']

yeo_xyz = {}
M = df_yeo_atlas.affine[:3, :3]
abc = df_yeo_atlas.affine[:3, 3]
for yeo_id in yeo_roi:
    yeo_ijk = np.array(np.nonzero(yeo_matr == yeo_id)).T
    yeo_xyz[yeo_id] = M.dot(yeo_ijk.T).T + abc.T
    
    
df_laus_atlas = nib.load('{}/Lausanne/ROIv_scale125_dilated.nii.gz'.format(path_AtlasData))
laus_matr = df_laus_atlas.get_data()
laus_roi = np.unique(laus_matr)[1:]

laus_xyz = {}
M = df_laus_atlas.affine[:3, :3]
abc = df_laus_atlas.affine[:3, 3]
for laus_id in laus_roi:
    laus_ijk = np.array(np.nonzero(laus_matr == laus_id)).T    
    laus_xyz[laus_id] = M.dot(laus_ijk.T).T + abc.T
    

laus_yeo_assign = []
for laus_id in laus_roi:
    dists = []

    for yeo_id in yeo_roi:
        dists.append(np.min(np.sum((yeo_xyz[yeo_id] - laus_xyz[laus_id].mean(axis=0))**2, axis=1)))
        
    laus_yeo_assign.append(yeo_names[np.argmin(dists)])
laus_yeo_assign = np.array(laus_yeo_assign)

pd.DataFrame(laus_yeo_assign).to_csv('{}/Lausanne/ROIv_scale125_dilated.Yeo2011_7Networks_MNI152.csv'.format(path_AtlasData))

# Manually replaced subcortical and cerebellar structures as SUB and CBR, respectively.

## System-Level Adjacency Matrices

In [ ]:
# Read in Yeo Atlas
df_laus_yeo = pd.read_csv('{}/LausanneScale125.csv'.format(path_CoreData))
df_laus_yeo = df_laus_yeo[df_laus_yeo.Label_ID != bad_roi[0]+1]
system_lbl = np.array(df_laus_yeo['Yeo2011_7Networks'].as_matrix())
system_name = np.unique(df_laus_yeo['Yeo2011_7Networks'])
n_system = len(system_name)
n_roi = len(system_lbl)
triu_ix, triu_iy = np.triu_indices(n_roi, k=1)
sys_triu_ix, sys_triu_iy = np.triu_indices(n_system, k=0)

# Reorder System Labels and Count ROIs per System
system_srt_ix = np.argsort(system_lbl)
system_cnt = np.array([len(np.flatnonzero(system_lbl == sys_name))
                       for sys_name in system_name])
system_demarc = np.concatenate(([0], np.cumsum(system_cnt)))

### Plot Population Average Adjacency Matrices (Expr + Pos/Neg)

In [ ]:
plt.figure(figsize=(5,5));
cnt = 0
for expr_id in ['Stroop', 'Navon']:
    df_list = glob.glob('{}/*.{}.npz'.format(path_ExpData, expr_id))
    
    for cor_id in ['pos', 'neg']:
        adj_pop = np.array([
            np.array([np.load(df_subj)['adj_dict'][()][cnd_id][tsk_id][cor_id].mean(axis=0)
                      for cnd_id in ['lo', 'hi']
                      for tsk_id in ['fix', 'task']]).mean(axis=0)
            for df_subj in df_list])

        adj_pop_mean = adj_pop.mean(axis=0)
        adj_pop_std = adj_pop.std(axis=0)     
        
        ax = plt.subplot(2, 2, cnt+1)
        mat = ax.matshow(adj_pop_mean[system_srt_ix, :][:, system_srt_ix],
                         cmap='magma');
        plt.colorbar(mat, ax=ax, fraction=0.046, pad=0.04)

        for xx in system_demarc:
            ax.vlines(xx, 0, n_roi, color='w', lw=0.5)
            ax.hlines(xx, 0, n_roi, color='w', lw=0.5)

        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_tick_params(width=0)                                
        ax.xaxis.set_tick_params(width=0)
        ax.grid(False)
        ax.tick_params(axis='both', which='major', pad=-3)

        ax.set_xticks((system_demarc[:-1] + (np.diff(system_demarc) * 0.5)));
        ax.set_xticklabels(system_name, fontsize=5.0, rotation=45)

        ax.set_yticks((system_demarc[:-1] + (np.diff(system_demarc) * 0.5)));
        ax.set_yticklabels(system_name, fontsize=5.0, rotation=45)

        ax.set_title('{}-{}'.format(expr_id, cor_id), fontsize=5.0)
        cnt += 1 
plt.show()

### Construct System Adjacency Matrices

In [ ]:
system_adj = {}    
for expr_id in ['Stroop', 'Navon']:
    system_adj[expr_id] = {}

    df_list = glob.glob('{}/*.{}.npz'.format(path_ExpData, expr_id))
    for df_subj in df_list:
        subj_id = int(df_subj.split('/')[-1].split('.')[0].split('_')[1])
        adj_subj_dict = np.load(df_subj)['adj_dict'][()]

        for cnd_id in ['lo', 'hi']:
            if cnd_id not in system_adj[expr_id]:
                system_adj[expr_id][cnd_id] = {}

            for tsk_id in ['fix', 'task']:
                if tsk_id not in system_adj[expr_id][cnd_id]:
                    system_adj[expr_id][cnd_id][tsk_id] = {}

                for cor_id in ['pos', 'neg']:  
                    if cor_id not in system_adj[expr_id][cnd_id][tsk_id].keys():
                        system_adj[expr_id][cnd_id][tsk_id][cor_id] = {}
                        
                    adj_sys_subj = []
                    for adj_matr in adj_subj_dict[cnd_id][tsk_id][cor_id]:
                        cfg_vec = convert_adj_matr_to_cfg_matr(adj_matr.reshape(-1, n_roi, n_roi))[0, ...]
                        
                        # Compute Brain System Adjacency Matrices
                        adj_sys = np.zeros((n_system, n_system))
                        for sys_ix, sys_iy in zip(*np.triu_indices(n_system, k=0)):
                            sys1 = system_name[sys_ix]
                            sys2 = system_name[sys_iy]
                            sys1_ix = np.flatnonzero(system_lbl[triu_ix] == sys1)
                            sys2_iy = np.flatnonzero(system_lbl[triu_iy] == sys2)
                            inter_sys_ii = np.intersect1d(sys1_ix, sys2_iy)
                            if len(inter_sys_ii) == 0:
                                sys1_ix = np.flatnonzero(system_lbl[triu_ix] == sys2)
                                sys2_iy = np.flatnonzero(system_lbl[triu_iy] == sys1)
                                inter_sys_ii = np.intersect1d(sys1_ix, sys2_iy)

                            mean_conn_sys1_sys2 = np.mean(cfg_vec[inter_sys_ii])

                            adj_sys[sys_ix, sys_iy] = mean_conn_sys1_sys2
                            
                        adj_sys += adj_sys.T
                        adj_sys_subj.append(adj_sys) 
                    system_adj[expr_id][cnd_id][tsk_id][cor_id][subj_id] = np.array(adj_sys_subj)
                    
np.savez('{}/Full_Adj.Yeo2011_7Networks.npz'.format(path_ExpData), system_adj=system_adj)

## Within/Between Task Contrasts

In [ ]:
system_adj = np.load('{}/Full_Adj.Yeo2011_7Networks.npz'.format(path_ExpData))['system_adj'][()]
subj_id = system_adj['Stroop']['hi']['fix']['pos'].keys()


# Within Task Differences
plt.figure(figsize=(3,3))
cnt = 0
print('*** Within Task Differences ***')
for expr_id in ['Stroop', 'Navon']:
    for cor_id in ['pos', 'neg']:
        
        cfg_sys_pop_hi = np.array([system_adj[expr_id]['hi']['task'][cor_id][s_id].mean(axis=0)
                                   for s_id in subj_id])[:, sys_triu_ix, sys_triu_iy].T
        
        cfg_sys_pop_lo = np.array([system_adj[expr_id]['lo']['task'][cor_id][s_id].mean(axis=0)
                                   for s_id in subj_id])[:, sys_triu_ix, sys_triu_iy].T
        
        cfg_sys_pop_cohd = []
        for cfg_hi, cfg_lo in zip(cfg_sys_pop_hi, cfg_sys_pop_lo):
            mean_hi = np.mean(cfg_hi)
            mean_lo = np.mean(cfg_lo)
            stdev = np.sqrt((np.std(cfg_hi)**2 + np.std(cfg_lo)**2) / 2)
            cohd = (mean_hi - mean_lo) / stdev
                        
            cfg_sys_pop_cohd.append(cohd)
        cfg_sys_pop_cohd = np.array(cfg_sys_pop_cohd)
        
        adj_sys_pop_cohd = np.zeros((n_system, n_system))
        adj_sys_pop_cohd[sys_triu_ix, sys_triu_iy] = cfg_sys_pop_cohd
        adj_sys_pop_cohd += np.triu(adj_sys_pop_cohd, k=1).T
        
        
        # Plot Heatmap
        ax = plt.subplot(2,2,cnt+1)
        vmin, vmax = [-np.max(np.abs(cfg_sys_pop_cohd)),
                      np.max(np.abs(cfg_sys_pop_cohd))]        
        mat = ax.matshow(adj_sys_pop_cohd, cmap='coolwarm',
                         vmin=vmin, vmax=vmax)
        plt.colorbar(mat, ax=ax, fraction=0.046, pad=0.04)

        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_tick_params(width=0)                                
        ax.xaxis.set_tick_params(width=0)
        ax.grid(False)
        ax.tick_params(axis='both', which='major', pad=-3)

        ax.set_xticks(np.arange(n_system));
        ax.set_yticks(np.arange(n_system));            
        ax.set_xticklabels(system_name, fontsize=3.0, rotation=45)
        ax.set_yticklabels(system_name, fontsize=3.0, rotation=45)

        ax.set_title('{}-{} :: Hi vs. Lo'.format(expr_id, cor_id), fontsize=5.0)
        cnt+=1
plt.show()        

        
# Between Task Differences
plt.figure(figsize=(3,3))
cnt = 0
print('*** Between Task Differences ***')
for cor_id in ['pos', 'neg']:
    for cnd_id in ['lo', 'hi']:
        
        cfg_sys_pop_strp = np.array([system_adj['Stroop'][cnd_id]['task'][cor_id][s_id].mean(axis=0)
                                       for s_id in subj_id])[:, sys_triu_ix, sys_triu_iy].T
        
        cfg_sys_pop_navn = np.array([system_adj['Navon'][cnd_id]['task'][cor_id][s_id].mean(axis=0)
                                     for s_id in subj_id])[:, sys_triu_ix, sys_triu_iy].T
        
        cfg_sys_pop_cohd = []
        for cfg_strp, cfg_navn in zip(cfg_sys_pop_strp, cfg_sys_pop_navn):
            mean_strp = np.mean(cfg_strp)
            mean_navn = np.mean(cfg_navn)
            stdev = np.sqrt((np.std(cfg_strp)**2 + np.std(cfg_navn)**2) / 2)
            cohd = (mean_strp - mean_navn) / stdev
                        
            cfg_sys_pop_cohd.append(cohd)
        cfg_sys_pop_cohd = np.array(cfg_sys_pop_cohd)
        
        adj_sys_pop_cohd = np.zeros((n_system, n_system))
        adj_sys_pop_cohd[sys_triu_ix, sys_triu_iy] = cfg_sys_pop_cohd
        adj_sys_pop_cohd += np.triu(adj_sys_pop_cohd, k=1).T
        
        
        # Plot Heatmap
        ax = plt.subplot(2,2,cnt+1)
        vmin, vmax = [-np.max(np.abs(cfg_sys_pop_cohd)),
                      np.max(np.abs(cfg_sys_pop_cohd))]        
        mat = ax.matshow(adj_sys_pop_cohd, cmap='coolwarm',
                         vmin=vmin, vmax=vmax)
        plt.colorbar(mat, ax=ax, fraction=0.046, pad=0.04)

        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_tick_params(width=0)                                
        ax.xaxis.set_tick_params(width=0)
        ax.grid(False)
        ax.tick_params(axis='both', which='major', pad=-3)

        ax.set_xticks(np.arange(n_system));
        ax.set_yticks(np.arange(n_system));            
        ax.set_xticklabels(system_name, fontsize=3.0, rotation=45)
        ax.set_yticklabels(system_name, fontsize=3.0, rotation=45)

        ax.set_title('{}-{} :: Stroop vs. Navon'.format(cnd_id, cor_id), fontsize=5.0)
        cnt+=1
plt.show()     

## Task Performance

In [ ]:
system_adj = np.load('{}/Full_Adj.Yeo2011_7Networks.npz'.format(path_ExpData))['system_adj'][()]
subj_id = system_adj['Stroop']['hi']['fix']['pos'].keys()

for perf_id in ['accuracy', 'medianRT']:
    
    plt.figure(figsize=(3, 3))
    cnt = 0
    for expr_id in ['Stroop', 'Navon']:
        for cor_id in ['pos', 'neg']:
        
            adj_sys_perf_matr = np.nan*np.zeros((n_system, n_system))

            for sys_ix in xrange(n_system):
                for sys_iy in xrange(n_system):
                    
                    edge = []
                    perf = []
                    for s_id in subj_id:
                        adj_hi = system_adj[expr_id]['hi']['task'][cor_id][s_id][:, sys_ix, sys_iy]
                        adj_lo = system_adj[expr_id]['lo']['task'][cor_id][s_id][:, sys_ix, sys_iy]
                        
                        edge.append(adj_hi.mean(axis=0) - adj_lo.mean(axis=0))
                        perf.append(df_perf[expr_id]['hi'][perf_id][s_id, :].mean() - df_perf[expr_id]['lo'][perf_id][s_id, :].mean())
                    edge = np.array(edge)
                    perf = np.array(perf)
                    
                    # Regress motion
                    m, yint, _, _, _ = stats.linregress(df_motion[expr_id], edge)
                    edge = edge - (m*df_motion[expr_id] + yint)

                    m, yint, _, _, _ = stats.linregress(df_motion[expr_id], perf)
                    perf = perf - (m*df_motion[expr_id] + yint)
                    
                    rv, pv = stats.pearsonr(edge, perf)
                    if pv < 0.05:
                        adj_sys_perf_matr[sys_ix, sys_iy] = rv
            
            # Plot Heatmap
            ax = plt.subplot(2, 2, cnt+1)
            vmin, vmax = [-np.nanmax(np.abs(adj_sys_perf_matr)),
                          np.nanmax(np.abs(adj_sys_perf_matr))]
            mat = ax.matshow(adj_sys_perf_matr, vmin=vmin, vmax=vmax, cmap='coolwarm')
            plt.colorbar(mat, ax=ax, fraction=0.046, pad=0.04)
            
            ax.yaxis.set_ticks_position('left')
            ax.xaxis.set_ticks_position('bottom')
            ax.yaxis.set_tick_params(width=0)                                
            ax.xaxis.set_tick_params(width=0)
            ax.grid(False)
            ax.tick_params(axis='both', which='major', pad=-3)

            ax.set_xticks(np.arange(n_system));
            ax.set_yticks(np.arange(n_system));            
            ax.set_xticklabels(system_name, fontsize=3.0, rotation=45)
            ax.set_yticklabels(system_name, fontsize=3.0, rotation=45)

            ax.set_title('{}-{} :: {}'.format(expr_id, cor_id, perf_id), fontsize=5.0)
            cnt += 1
    plt.show()